In [14]:
### Libraries.
import matplotlib, matplotlib.pyplot as plt
import numpy  as np
import pandas as pd
import pymc
% matplotlib inline
import seaborn as sns
sns.set_style('white')
from sklearn.cross_validation import KFold

In [3]:
# Games path.
games_path = 'Data/games.csv'
# Read games.
games = pd.read_csv(games_path)
# Trim to single group.
games = games[games.game_group == 1].reset_index(drop=True)
# 2014 games.
games_2014 = games[games.year == 2014].reset_index(drop=True)

In [4]:
"""
Inputs:
    data: 
    features: 
    coef_dists: 
    b0_params: 
    err_params:
    default_coef_dist: Coefficient distributions.
    default_coef_params: Coefficient distribution parameters.
    step_method: MCMC stepping method.
    step_method_params: Parameters for stepping method. 
"""
def model_games (
    data=games,
    features=['team_Pythag'],
    coef_dists=None,
    b0_params={'mu':0, 'tau':0.0003, 'value':0},
    err_params=[.5],
    default_coef_dist = pymc.Normal,
    default_coef_params = {'mu':0, 'tau':0.0003, 'value':0},
    step_method = pymc.Metropolis,
    step_method_params = {'proposal_sd':1., 'proposal_distribution':'Normal'}
):
    
    # Define priors on intercept and error. PyMC uses precision (inverse variance).
    b0 = pymc.Normal('b_0', **b0_params)
    err = pymc.Bernoulli('err', *err_params)
    
    # Containers for coefficients and data.
    b = np.empty(len(features), dtype=object)
    x = np.empty(len(features), dtype=object)
    
    # Traverse features.
    for i, f in enumerate(features):
        # Coefficient.
        # First start with the distribution. Use one if we've been given one; else use default.
        coef_dist_type = default_coef_dist if coef_dists is None or coef_dists[i] is None or coef_dists[i][0] is None else coef_dists[i][0]
        # Now handle parameters.
        coef_dist_params = default_coef_params if coef_dists is None or coef_dists[i] is None or coef_dists[i][1] is None else coef_dists[i][1]
        # Now actually create the coefficient distribution
        b[i] = coef_dist_type('b_'+f, **coef_dist_params)
        # Data distribution.
        x[i] = pymc.Normal('x_'+f, 0, 1, value=np.array(data[f]), observed=True)
    
    # Logistic function.
    @pymc.deterministic
    def logistic(b0=b0, b=b, x=x):
        return 1.0 / (1. + np.exp(-(b0 + b.dot(x))))

    # Get outcome data.
    y = np.array(data.win)
    # Model outcome as a Bernoulli distribution.
    y = pymc.Bernoulli('win', logistic, value=y, observed=True)
    
    # Define model, MCMC object.
    model = pymc.Model([logistic, pymc.Container(b), err, pymc.Container(x), y])
    mcmc  = pymc.MCMC(model)
    
    # Configure step methods.
    for var in list(b)+[b0,err]:
        mcmc.use_step_method(step_method, stochastic=var, **step_method_params)
    
    # Return MCMC object.
    return mcmc

In [5]:
# Features.
features = ['location_Home','diff_Tempo','diff_OE','diff_DE','diff_Pythag']
model_mcmc = model_games(data=games_2014,features=features)

In [6]:
model_mcmc.sample(10000,2000)

 [-----------------100%-----------------] 10000 of 10000 complete in 14.4 sec

In [7]:
print model_mcmc.trace('b_0')[:].mean()
print model_mcmc.trace('b_diff_Pythag')[:].mean()

-0.446862851864
6.39136864289


In [25]:
"""
Inputs:
    data:       Dataframe with game data.
    model_mcmc: PyMC MCMC object.
    features:   list of features in data dataframe.
    method:     'pp' for posterior predictive, 'map' for single estimate using MAP.
"""
def predict_games(data, model_mcmc, features, method='pp'):
    # Trace and feature length.
    trace_len = model_mcmc.trace('b_0')[:].shape[0]
    coef_samples = trace_len
    feat_len  = len(features)+1
    data_len  = len(data)
    
    # Get coefficients.
    coefs = np.empty((trace_len,feat_len))
    for f_i, f in enumerate(['0']+features):
        coefs[:,f_i] = model_mcmc.trace('b_'+f)[:]
    # Use MAP if we need to.
    if method == 'map':
        coefs = coefs.mean(axis=0).reshape((1,feat_len))
        coef_samples = 1
    
    # Get design matrix.
    X = np.ones((data_len,feat_len))
    for f_i, f in enumerate(features):
        X[:,f_i+1] = data.ix[:,f]
    # Get wins.
    y = np.array(data.win)
    
    # Logistic function for win calculation.
    logistic = lambda s: 1 / (1+np.exp(-s))
    
    # Estimate wins. Start with a wins container.
    y_hat_raw = np.zeros((coef_samples,data_len))
    y_hat     = np.zeros((coef_samples,data_len), dtype=np.int)
    # Traverse set of coefficients and assemble win estimates.
    for c_i, c in enumerate(coefs):
        y_hat_raw[c_i] = logistic((c.T*X).sum(axis=1))
    # Tag wins/losses with comparison.
    y_hat = (y_hat_raw >= .5).astype(np.int)
    
    # Compute accuracy by wins.
    y_hat_accuracy = (y == y_hat).mean()
    
    return y_hat_raw, y_hat, y_hat_accuracy

In [40]:
# function to do cross validation
def mcmc_xval(X, features, K):
    scores = []
    kf = KFold(len(X), 5,shuffle=True)
    for train, test in kf:
        X_train = X.ix[train]
        X_test = X.ix[test]
        model_mcmc = model_games(data=X_train,features=features)
        model_mcmc.sample(10000,2000)
        y_hat_raw, y_hat, y_hat_accuracy = predict_games(X_test, model_mcmc, features, 'pp')
        scores.append(y_hat_accuracy)
    return np.mean(np.array(scores))

In [ ]:
mcmc_xval(games_2014,features,5)

 [-----------      31%                  ] 3149 of 10000 complete in 4.0 secHalting at iteration  3180  of  10000
 [-----------------62%---               ] 6225 of 10000 complete in 8.0 sec